**Packages**

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 

from datasets import load_dataset
import evaluate

#let's make longer output readable without horizontal scrolling
from pprint import pprint

import warnings

/home/ubuntu/w266/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-03-27 21:44:52.996106: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-03-27 21:44:57.490615: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/amazon/efa/lib:/opt/amazon/openmpi/lib:/usr/local/cuda/efa/lib:/usr/local/cuda/lib:/usr/local/cuda:/usr/local/cuda/lib64:/usr/local/cuda/extras/C

**Necessary Functions**

In [2]:
import inspect

def get_default_args(func):
    signature = inspect.signature(func)
    return {
        k: v.default
        for k, v in signature.parameters.items()
        if v.default is not inspect.Parameter.empty
    }

In [3]:
rouge = evaluate.load('rouge')

**Data**

In [4]:
dataset = load_dataset("csebuetnlp/xlsum", "english")

Found cached dataset xlsum (/home/ubuntu/.cache/huggingface/datasets/csebuetnlp___xlsum/english/2.0.0/518ab0af76048660bcc2240ca6e8692a977c80e384ffb18fdddebaca6daebdce)
100%|█████████████████████████████████████████████| 3/3 [00:00<00:00,  5.46it/s]


In [5]:
# EDA
len(dataset['train'])

306522

In [6]:
# EDA
dataset['train'][1]

{'id': 'uk-scotland-highlands-islands-11069985',
 'url': 'https://www.bbc.com/news/uk-scotland-highlands-islands-11069985',
 'title': 'Huge tidal turbine installed at Orkney test site',
 'summary': 'The massive tidal turbine AK1000 has been installed in 35m (114.8ft) of water at a test site in Orkney.',
 'text': 'Atlantis Resources unveiled the marine energy device at Invergordon ahead of it being shipped to Kirkwall. Trials on the device will now be run at the European Marine Energy Centre test site off Eday. The device stands 22.5m (73ft) tall, weighs 1,300 tonnes and has two sets of blades on a single unit. It could generate enough power for 1,000 homes.'}

In [7]:
index = pd.DataFrame({"index": list(range(len(dataset['train'])))})
sample_index = index.sample(n=2000, replace=False, random_state=1004)
sample_index[:5]

,index
235420,235420
172024,172024
253546,253546
224954,224954
214134,214134


In [8]:
id = []
url = []
title = []
article = []
article_num_sentences = []
article_num_characters = []
summary = []
summary_num_sentences = []
summary_num_characters = []

for i in sample_index["index"]:
    id.append(dataset["train"][i]['id'])
    url.append(dataset["train"][i]['url'])
    title.append(dataset["train"][i]['title'])
    summary.append(dataset["train"][i]['summary'])
    summary_num_sentences.append(len(dataset["train"][i]['summary'].split(".")))
    summary_num_characters.append(len(dataset["train"][i]['summary']))
    article.append(dataset["train"][i]['text'])
    article_num_sentences.append(len(dataset["train"][i]['text'].split(".")))
    article_num_characters.append(len(dataset["train"][i]['text']))

In [9]:
d = {'id': id, 'url': url, "title": title, 'article': article, "article_num_sentences": article_num_sentences, "article_num_characters": article_num_characters, 'summary': summary,"summary_num_sentences": summary_num_sentences, "summary_num_characters": summary_num_characters}
df = pd.DataFrame(data=d)
df.head(5)

,id,url,title,article,article_num_sentences,article_num_characters,summary,summary_num_sentences,summary_num_characters
0,uk-england-cornwall-55191422,https://www.bbc.com/news/uk-england-cornwall-5...,Care home manager: 'It felt like we were losin...,By Rebecca Ricks & Johnny O'SheaBBC Spotlight ...,37,3755,"During the spring, at the height of the Covid-...",2,147
1,uk-43893709,https://www.bbc.com/news/uk-43893709,Tafida Raqeeb: Who decides the care of sick ch...,"By Rachel SchraerBBC Reality Check So, why did...",33,4531,"The parents of five-year-old Tafida Raqeeb, wh...",2,121
2,uk-politics-57050659,https://www.bbc.com/news/uk-politics-57050659,Labour reshuffle: Anneliese Dodds out in Starm...,Anneliese Dodds will now become the Labour Par...,36,4845,Sir Keir Starmer has sacked his shadow chancel...,2,115
3,entertainment-arts-38221420,https://www.bbc.com/news/entertainment-arts-38...,Vinyl sales made more than downloads last week,By Mark SavageBBC Music reporter Vinyl sales m...,27,2082,More money was spent on vinyl than downloaded ...,2,83
4,entertainment-arts-24046991,https://www.bbc.com/news/entertainment-arts-24...,Pirates of the Caribbean sequel delayed,Disney's Pirates of The Caribbean: Dead Men Te...,14,1569,The next Pirates of the Caribbean film has bee...,2,88


**BERT Extractive Model**

Model 0: Default Hyperparameters

In [33]:
warnings.filterwarnings("ignore")

from summarizer import Summarizer

model = Summarizer()

loading configuration file config.json from cache at /home/ubuntu/.cache/huggingface/hub/models--bert-large-uncased/snapshots/80792f8e8216b29f3c846b653a0ff0a37c210431/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.27.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /home/ubuntu/.cache/huggingface/hub/models--bert-large-uncased/snapshots/80792f8e8216b29f3c846b653a0ff0a37c210431/

In [34]:
import warnings
warnings.filterwarnings("ignore")

bert_r1 = []
bert_r2 = []
bert_rL = []
bert_rLs = []

for i in range(int(len(df['article'])/2)):
    
    candidate = model(df['article'][i],
                     )
    candidate = [candidate]
    #pprint(candidate[0], compact=True)
    
    ref = [df['summary'][i]]
    
    results = rouge.compute(predictions=candidate,
                            references=ref)
    
    bert_r1.append(results['rouge1'])
    bert_r2.append(results['rouge2'])
    bert_rL.append(results['rougeL'])
    bert_rLs.append(results['rougeLsum'])
    
    if i in np.arange(0, (len(df['article']) + 101), 100):
        data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs}
        scores = pd.DataFrame(data)
        scores.to_csv(r'BERT_0_scores.csv', index=False)
        print(i)

data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs}
scores = pd.DataFrame(data)
scores.to_csv(r'BERT_0_scores.csv', index=False)
print(i)

0
100
200
300
400
500
600
700
800
900
999


In [35]:
print('rouge1 average :', np.mean(bert_r1))
print('rouge2 average :', np.mean(bert_r2))
print('rougeL average :', np.mean(bert_rL))
print('rougeLs average :', np.mean(bert_rLs))

rouge1 average : 0.1555884135260613
rouge2 average : 0.02396541867504151
rougeL average : 0.1019488717714199
rougeLs average : 0.1019488717714199


Finetuning On First 1000

Model 1. Adjusted min_length, max_length, num_sentences (which overrides ratio)

In [50]:
from summarizer import Summarizer

In [51]:
get_default_args(Summarizer)

{'model': 'bert-large-uncased',
 'custom_model': None,
 'custom_tokenizer': None,
 'hidden': -2,
 'reduce_option': 'mean',
 'sentence_handler': <summarizer.sentence_handler.SentenceHandler at 0x7ff4f591a2e0>,
 'random_state': 12345,
 'hidden_concat': False}

In [52]:
# Using bert-base instead of bert-large to reduce run times
model = Summarizer(model='bert-base-uncased')

loading configuration file config.json from cache at /home/ubuntu/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.27.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /home/ubuntu/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/pyt

In [53]:
get_default_args(model)

{'ratio': 0.2,
 'min_length': 40,
 'max_length': 600,
 'use_first': True,
 'algorithm': 'kmeans',
 'num_sentences': None,
 'return_as_list': False}

In [54]:
import warnings
warnings.filterwarnings("ignore")

bert_r1 = []
bert_r2 = []
bert_rL = []
bert_rLs = []

for i in range(int(len(df['article'])/2)):
    
    # Limiting number sentences in each summary generated to 2 sentences 
    candidate = model(df['article'][i], 
                      num_sentences = round(df["summary_num_sentences"][:1000].mean()), 
                      min_length = min(df["summary_num_characters"][:1000]),
                      max_length = max(df["summary_num_characters"][:1000]),        
                      ratio = None,
                      use_first = None,
                     )
    candidate = [candidate]
    #pprint(candidate[0], compact=True)
    
    ref = [df['summary'][i]]
    
    results = rouge.compute(predictions=candidate,
                            references=ref)
    
    bert_r1.append(results['rouge1'])
    bert_r2.append(results['rouge2'])
    bert_rL.append(results['rougeL'])
    bert_rLs.append(results['rougeLsum'])
    
    if i in np.arange(0, (len(df['article']) + 101), 100):
        data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs}
        scores = pd.DataFrame(data)
        scores.to_csv(r'BERT_1_scores.csv', index=False)
        print(i)
        
data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs}
scores = pd.DataFrame(data)
scores.to_csv(r'BERT_1_scores.csv', index=False)
print(i)

0
100
200
300
400
500
600
700
800
900


In [55]:
print('rouge1 average :', np.mean(bert_r1))
print('rouge2 average :', np.mean(bert_r2))
print('rougeL average :', np.mean(bert_rL))
print('rougeLs average :', np.mean(bert_rLs))

rouge1 average : 0.178192403298457
rouge2 average : 0.026733936113222955
rougeL average : 0.11966303460985027
rougeLs average : 0.11966303460985027


Tuning Hyperparameters

2. Made num_sentences based off number of clusters instead of average 

In [10]:
warnings.filterwarnings("ignore")

from summarizer import Summarizer

model = Summarizer(model='bert-base-uncased',
                  )

2023-03-27 21:45:40.058167: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2023-03-27 21:45:40.058228: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ip-172-31-55-240): /proc/driver/nvidia/version does not exist
loading configuration file config.json from cache at /home/ubuntu/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
 

In [11]:
warnings.filterwarnings("ignore")

bert_r1 = []
bert_r2 = []
bert_rL = []
bert_rLs = []

for i in range(int(len(df['article'])/2)):

    res = model.calculate_optimal_k(df['article'][i], k_max=10)
    
    candidate = model(df['article'][i], 
                      num_sentences = res, # number of sentences determined by number of clusters
                      min_length = min(df["summary_num_characters"][:1000]),
                      max_length = max(df["summary_num_characters"][:1000]),        
                      ratio = None,
                      use_first = None,)
    candidate = [candidate]
    #pprint(candidate[0], compact=True)
    
    ref = [df['summary'][i]]
    
    results = rouge.compute(predictions=candidate,
                            references=ref)
    
    bert_r1.append(results['rouge1'])
    bert_r2.append(results['rouge2'])
    bert_rL.append(results['rougeL'])
    bert_rLs.append(results['rougeLsum'])
    
    if i in np.arange(0, (len(df['article']) + 101), 100):
        data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs}
        scores = pd.DataFrame(data)
        scores.to_csv(r'BERT_2_scores.csv', index=False)
        print(i)
        
data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs}
scores = pd.DataFrame(data)
scores.to_csv(r'BERT_2_scores.csv', index=False)
print(i)

0
100
200
300
400
500
600
700
800
900


In [12]:
print('rouge1 average :', np.mean(bert_r1))
print('rouge2 average :', np.mean(bert_r2))
print('rougeL average :', np.mean(bert_rL))
print('rougeLs average :', np.mean(bert_rLs))

rouge1 average : 0.16466988117522663
rouge2 average : 0.026490759507054564
rougeL average : 0.1089025377167142
rougeLs average : 0.1089025377167142


3. Setting number of sentences with clusters didn't help in second model, so we are reverting back to first model but setting use_first = False bc many examples have meta data in first sentence.

In [13]:
from summarizer import Summarizer

warnings.filterwarnings("ignore")

model = Summarizer(model='bert-base-uncased',
                  )

loading configuration file config.json from cache at /home/ubuntu/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.27.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /home/ubuntu/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/pyt

In [16]:
import warnings
warnings.filterwarnings("ignore")

bert_r1 = []
bert_r2 = []
bert_rL = []
bert_rLs = []

for i in range(int(len(df['article'])/2)):
    
    # Limiting number sentences in each summary generated to 2 sentences 
    candidate = model(df['article'][i], 
                      num_sentences = round(df["summary_num_sentences"][:1000].mean()), 
                      min_length = min(df["summary_num_characters"][:1000]),
                      max_length = max(df["summary_num_characters"][:1000]),        
                      ratio = None,
                      use_first = False,
                     )
    candidate = [candidate]
    #pprint(candidate[0], compact=True)
    
    ref = [df['summary'][i]]
    
    results = rouge.compute(predictions=candidate,
                            references=ref)
    
    bert_r1.append(results['rouge1'])
    bert_r2.append(results['rouge2'])
    bert_rL.append(results['rougeL'])
    bert_rLs.append(results['rougeLsum'])
    
    if i in np.arange(0, (len(df['article']) + 101), 100):
        data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs}
        scores = pd.DataFrame(data)
        scores.to_csv(r'BERT_3_scores.csv', index=False)
        print(i)

data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs}
scores = pd.DataFrame(data)
scores.to_csv(r'BERT_3_scores.csv', index=False)
print(i)

0
100
200
300
400
500
600
700
800
900


In [17]:
print('rouge1 average :', np.mean(bert_r1))
print('rouge2 average :', np.mean(bert_r2))
print('rougeL average :', np.mean(bert_rL))
print('rougeLs average :', np.mean(bert_rLs))

rouge1 average : 0.178192403298457
rouge2 average : 0.026733936113222955
rougeL average : 0.11966303460985027
rougeLs average : 0.11966303460985027


4. Setting use_first = True

In [19]:
warnings.filterwarnings("ignore")

from summarizer import Summarizer

model = Summarizer(model='bert-base-uncased',
                  )

loading configuration file config.json from cache at /home/ubuntu/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.27.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /home/ubuntu/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/pyt

In [20]:
import warnings
warnings.filterwarnings("ignore")

bert_r1 = []
bert_r2 = []
bert_rL = []
bert_rLs = []

for i in range(int(len(df['article'])/2)):
    
    # Limiting number sentences in each summary generated to 2 sentences 
    candidate = model(df['article'][i], 
                      num_sentences = round(df["summary_num_sentences"][:1000].mean()), 
                      min_length = min(df["summary_num_characters"][:1000]),
                      max_length = max(df["summary_num_characters"][:1000]),        
                      ratio = None,
                      use_first = True,
                     )
    candidate = [candidate]
    #pprint(candidate[0], compact=True)
    
    ref = [df['summary'][i]]
    
    results = rouge.compute(predictions=candidate,
                            references=ref)
    
    bert_r1.append(results['rouge1'])
    bert_r2.append(results['rouge2'])
    bert_rL.append(results['rougeL'])
    bert_rLs.append(results['rougeLsum'])
    
    if i in np.arange(0, (len(df['article']) + 101), 100):
        data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs}
        scores = pd.DataFrame(data)
        scores.to_csv(r'BERT_3_scores.csv', index=False)
        print(i)

data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs}
scores = pd.DataFrame(data)
scores.to_csv(r'BERT_3_scores.csv', index=False)
print(i)

0
100
200
300
400
500
600
700
800
900
999


In [21]:
print('rouge1 average :', np.mean(bert_r1))
print('rouge2 average :', np.mean(bert_r2))
print('rougeL average :', np.mean(bert_rL))
print('rougeLs average :', np.mean(bert_rLs))

rouge1 average : 0.1711426991129008
rouge2 average : 0.026139674733637642
rougeL average : 0.11264718864577299
rougeLs average : 0.11264718864577299


5. Use bert-large-uncased w/ best from above

In [22]:
warnings.filterwarnings("ignore")

from summarizer import Summarizer

model = Summarizer(model='bert-large-uncased',
                  )

loading configuration file config.json from cache at /home/ubuntu/.cache/huggingface/hub/models--bert-large-uncased/snapshots/80792f8e8216b29f3c846b653a0ff0a37c210431/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 1024,
  "initializer_range": 0.02,
  "intermediate_size": 4096,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 16,
  "num_hidden_layers": 24,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.27.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /home/ubuntu/.cache/huggingface/hub/models--bert-large-uncased/snapshots/80792f8e8216b29f3c846b653a0ff0a37c210431/

In [23]:
import warnings
warnings.filterwarnings("ignore")

bert_r1 = []
bert_r2 = []
bert_rL = []
bert_rLs = []

for i in range(int(len(df['article'])/2)):
    
    # Limiting number sentences in each summary generated to 2 sentences 
    candidate = model(df['article'][i], 
                      num_sentences = round(df["summary_num_sentences"][:1000].mean()), 
                      min_length = min(df["summary_num_characters"][:1000]),
                      max_length = max(df["summary_num_characters"][:1000]),        
                      ratio = None,
                      use_first = False,
                     )
    candidate = [candidate]
    #pprint(candidate[0], compact=True)
    
    ref = [df['summary'][i]]
    
    results = rouge.compute(predictions=candidate,
                            references=ref)
    
    bert_r1.append(results['rouge1'])
    bert_r2.append(results['rouge2'])
    bert_rL.append(results['rougeL'])
    bert_rLs.append(results['rougeLsum'])
    
    if i in np.arange(0, (len(df['article']) + 101), 100):
        data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs}
        scores = pd.DataFrame(data)
        scores.to_csv(r'BERT_4_scores.csv', index=False)
        print(i)

data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs}
scores = pd.DataFrame(data)
scores.to_csv(r'BERT_4_scores.csv', index=False)
print(i)

0
100
200
300
400
500
600
700
800
900
999


In [24]:
# this model took longer and performed worse
print('rouge1 average :', np.mean(bert_r1))
print('rouge2 average :', np.mean(bert_r2))
print('rougeL average :', np.mean(bert_rL))
print('rougeLs average :', np.mean(bert_rLs))

rouge1 average : 0.17709196861695112
rouge2 average : 0.025145083935109475
rougeL average : 0.11939202129428797
rougeLs average : 0.11939202129428797


6. Reduce option median

In [27]:
warnings.filterwarnings("ignore")

from summarizer import Summarizer

model = Summarizer(model='bert-base-uncased', reduce_option = 'median'
                  )

loading configuration file config.json from cache at /home/ubuntu/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.27.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /home/ubuntu/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/pyt

In [28]:
import warnings
warnings.filterwarnings("ignore")

bert_r1 = []
bert_r2 = []
bert_rL = []
bert_rLs = []

for i in range(int(len(df['article'])/2)):
    
    # Limiting number sentences in each summary generated to 2 sentences 
    candidate = model(df['article'][i], 
                      num_sentences = round(df["summary_num_sentences"][:1000].mean()), 
                      min_length = min(df["summary_num_characters"][:1000]),
                      max_length = max(df["summary_num_characters"][:1000]),        
                      ratio = None,
                      use_first = None,
                     )
    candidate = [candidate]
    #pprint(candidate[0], compact=True)
    
    ref = [df['summary'][i]]
    
    results = rouge.compute(predictions=candidate,
                            references=ref)
    
    bert_r1.append(results['rouge1'])
    bert_r2.append(results['rouge2'])
    bert_rL.append(results['rougeL'])
    bert_rLs.append(results['rougeLsum'])
    
    if i in np.arange(0, (len(df['article']) + 101), 100):
        data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs}
        scores = pd.DataFrame(data)
        scores.to_csv(r'BERT_4_scores.csv', index=False)
        print(i)

data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs}
scores = pd.DataFrame(data)
scores.to_csv(r'BERT_6_scores.csv', index=False)
print(i)

0
100
200
300
400
500
600
700
800
900
999


In [29]:
print('rouge1 average :', np.mean(bert_r1))
print('rouge2 average :', np.mean(bert_r2))
print('rougeL average :', np.mean(bert_rL))
print('rougeLs average :', np.mean(bert_rLs))

rouge1 average : 0.17604122934873215
rouge2 average : 0.025439640514237527
rougeL average : 0.11815625686666714
rougeLs average : 0.11815625686666714


7. Reduce option max THIS WORKED THIS BEST!!!

In [30]:
warnings.filterwarnings("ignore")

from summarizer import Summarizer

model = Summarizer(model='bert-base-uncased', reduce_option = 'max'
                  )

loading configuration file config.json from cache at /home/ubuntu/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/config.json
Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "output_hidden_states": true,
  "pad_token_id": 0,
  "position_embedding_type": "absolute",
  "transformers_version": "4.27.1",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 30522
}

loading weights file pytorch_model.bin from cache at /home/ubuntu/.cache/huggingface/hub/models--bert-base-uncased/snapshots/0a6aa9128b6194f4f3c4db429b6cb4891cdb421b/pyt

In [31]:
import warnings
warnings.filterwarnings("ignore")

bert_r1 = []
bert_r2 = []
bert_rL = []
bert_rLs = []

for i in range(int(len(df['article'])/2)):
    
    # Limiting number sentences in each summary generated to 2 sentences 
    candidate = model(df['article'][i], 
                      num_sentences = round(df["summary_num_sentences"][:1000].mean()), 
                      min_length = min(df["summary_num_characters"][:1000]),
                      max_length = max(df["summary_num_characters"][:1000]),        
                      ratio = None,
                      use_first = None,
                     )
    candidate = [candidate]
    #pprint(candidate[0], compact=True)
    
    ref = [df['summary'][i]]
    
    results = rouge.compute(predictions=candidate,
                            references=ref)
    
    bert_r1.append(results['rouge1'])
    bert_r2.append(results['rouge2'])
    bert_rL.append(results['rougeL'])
    bert_rLs.append(results['rougeLsum'])
    
    if i in np.arange(0, (len(df['article']) + 101), 100):
        data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs}
        scores = pd.DataFrame(data)
        scores.to_csv(r'BERT_7_scores.csv', index=False)
        print(i)

data = {'rouge1': bert_r1, 'rouge2': bert_r2, 'rogueL': bert_rL, 'rogueLs': bert_rLs}
scores = pd.DataFrame(data)
scores.to_csv(r'BERT_7_scores.csv', index=False)
print(i)

0
100
200
300
400
500
600
700
800
900
999


In [32]:
print('rouge1 average :', np.mean(bert_r1))
print('rouge2 average :', np.mean(bert_r2))
print('rougeL average :', np.mean(bert_rL))
print('rougeLs average :', np.mean(bert_rLs))

rouge1 average : 0.18111656560761913
rouge2 average : 0.025095740456991585
rougeL average : 0.12307647067888493
rougeLs average : 0.12307647067888493
